In [1]:
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))


Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3090


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360
predict_days = 30
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[1500, data_len - input_days - test_predict_len],
                        targetDays=5,
                        encoderDecoderParameter="close",
                        targetDataParameter="close")
print("Train days : ", train_dataSet.data.shape)
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len],
                             targetDays=5,
                             encoderDecoderParameter="close",
                             targetDataParameter="close")
print("Validation days ; ", validation_dataSet.data.shape)
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len],
                       targetDays=5,
                       encoderDecoderParameter="close",
                       targetDataParameter="close")
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

All days :  2994
Train days :  (1274, 1440, 10)
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)


In [4]:
from os.path import join
models_store_path = "../StockPriceForecast/Models/"
days_num = len(train_dataLoader)
loss_function_name = "softDTW"
lr = 0.0001
num_epochs = 1
model_name = f"spf_{loss_function_name}_{num_epochs}_{days_num}.pth"
model_store_path = join(models_store_path, model_name)
# train_dataSet = train_dataSet.isel(0, 390 + days_num -1)
# train_dataLoader = DataLoader(train_dataSet)
print(model_store_path)

../StockPriceForecast/Models/spf_softDTW_1_910.pth


In [5]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
model = Seq2SeqAttention(input_size=1440,
                         hidden_size=4096,
                         output_size=1440,
                         num_layers=3,
                         dropout=0.1)
# train_dataLoader.isel(startIndex=950, endIndex=len(train_dataLoader))
model, ls = train_Seq2SeqAttention(model=model,
                                   device=device,
                                   dataLoader=train_dataLoader,
                                   loss_function_name=loss_function_name,
                                   lr=lr,
                                   num_epochs=num_epochs)

epoch 0 training: 100%|██████████| 910/910 [11:23<00:00,  1.33it/s, loss=54, lr=1.95e-7]  


In [6]:
torch.save(model, model_store_path)

In [7]:
print(len(train_dataSet))
print(len(train_dataLoader))
print((len(train_dataLoader) + (360 + 30) - 1))

1324
960
1349


In [7]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
sys.path.append("../StockPriceForecast/")
model = torch.load("../StockPriceForecast/Models/spf_softDTW_1_910.pth")
validation_dataLoader.isel(0, 1)
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=5,
                                                    device=device)
print(len(predicts))

1


In [24]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
show_index = 1
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts[0][show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets[0][show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [9]:
import numpy as np
print(ls.__len__())
trace = dict(
    # x = np.linspace(0, len(ls), 1),
    y = np.array(ls),
    mode = "lines",
    name = "loss"
)
fig = go.Figure(trace)
fig.show()

910


1910
1010


In [23]:
# 继续训练
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
from os.path import join
models_store_path = "../StockPriceForecast/Models/"
# train_dataLoader.isel(900, len(train_dataLoader))
days_num = len(train_dataLoader)
loss_function_name = "softDTW"
lr = 0.0001
num_epochs = 3
model_name = f"spf_{loss_function_name}_{num_epochs}_{days_num}.pth"
model_store_path = join(models_store_path, model_name)
print(model_store_path)

../StockPriceForecast/Models/spf_softDTW_3_910.pth


In [24]:
model = torch.load("../StockPriceForecast/Models/spf_softDTW_1_910.pth")
model, ls = train_Seq2SeqAttention(model=model,
                                   device=device,
                                   dataLoader=train_dataLoader,
                                   loss_function_name=loss_function_name,
                                   init_weights=False,
                                   lr=lr,
                                   num_epochs=num_epochs)

epoch 2 training: 100%|██████████| 910/910 [07:58<00:00,  1.90it/s, loss=65.7, lr=1.22e-8]


In [28]:
torch.save(model, model_store_path)


In [4]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
model = torch.load("../StockPriceForecast/Models/spf_softDTW_1_910.pth")
sys.path.append("../StockPriceForecast/")
validation_dataLoader.isel(0, 1)
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=5,
                                                    device=device)
print(len(predicts))

1


In [26]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
show_index = 2
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts[0][show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets[0][show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [12]:
print(ls.__len__())
trace = dict(
    # x = np.linspace(0, len(ls), 1),
    y = np.array(ls),
    mode = "lines",
    name = "loss"
)
fig = go.Figure(trace)
fig.show()


3030
